In [2]:
import pandas as pd
import numpy as np
import json
from cold_start import get_cold_start_rating

In [7]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [8]:
ratings_df = spark.read.json('data/ratings.json').toPandas()
metadata = pd.read_csv('data/movies_metadata.csv')
request_df = spark.read.json('data/requests.json').toPandas()

/Users/nawjux/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
ratings_df['user_id'].nunique()

5400

In [17]:
ratings_df['rating'].value_counts()

4    250180
3    186675
5    166246
2     76138
1     40710
Name: rating, dtype: int64

In [15]:
ratings_df.isna().sum()

movie_id     0
rating       0
timestamp    0
user_id      0
dtype: int64

In [30]:
len(metadata), metadata['tagline'].isna().sum()

(45466, 25054)

In [34]:
metadata.loc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [4]:
len(requests_df)

NameError: name 'requests_df' is not defined

In [57]:
users = []
for line in open('data/users.dat', 'r'):
    item = line.split('\n')
    users.append(item[0].split("::"))

In [18]:
user_df = pd.read_csv('data/users.dat', sep='::', header=None, names=['id', 'gender', 'age', 'occupation', 'zip'])
movie_info_df = pd.read_csv('data/movies.dat', sep='::', header=None, names=['id', 'name', 'genres'])

/Users/nawjux/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/Users/nawjux/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [19]:
user_df[20:53]

,id,gender,age,occupation,zip
20,21,M,18,16,99353
21,22,M,18,15,53706
22,23,M,35,0,90049
23,24,F,25,7,10023
24,25,M,18,4,01609
25,26,M,25,7,23112
26,27,M,25,11,19130
27,28,F,25,1,14607
28,29,M,35,7,33407
29,30,F,35,7,19143


In [121]:
movie_info_df.head()

,id,name,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [122]:
movie_info_df['genres'] = movie_info_df['genres'].apply(lambda x: x.split('|'))

In [125]:
movie_info_df.head()

,id,name,genres
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [133]:
all_genres = set([item for movie in movie_info_df['genres'] for item in movie])

In [135]:
all_genres

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [20]:
user_df = user_df.drop('zip', axis=1)

In [21]:
user_df.head()

,id,gender,age,occupation
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20


In [24]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [25]:
def ohe_columns(series, name):
    ohe = OneHotEncoder(categories='auto')
    ohe.fit(series)
    cols = ohe.get_feature_names(name)
    ohe = ohe.transform(series)
    final_df = pd.DataFrame(ohe.toarray(), columns=cols)
    return final_df

In [26]:
# OHE the user cols
my_cols = ['gender', 'age', 'occupation']

ohe_multi = OneHotEncoder(categories='auto')
ohe_multi.fit(user_df[my_cols])
ohe_mat = ohe_multi.transform(user_df[my_cols])

In [27]:
# Then KMeans cluster
k_clusters = KMeans(n_clusters=8, random_state=42)
k_clusters.fit(ohe_mat)

preds = k_clusters.predict(ohe_mat)

In [28]:
preds

array([4, 0, 2, ..., 4, 4, 2], dtype=int32)

In [311]:
preds.shape

(6040,)

In [29]:
def add_clusters_to_users(n_clusters=8):
    """
    parameters:number of clusters
    
    return: user dataframe
    """
    
    # Get the user data
    user_df = pd.read_csv('data/users.dat', sep='::', header=None
                          , names=['id', 'gender', 'age', 'occupation', 'zip'])
    
    # OHE for clustering
    my_cols = ['gender', 'age', 'occupation']

    ohe_multi = OneHotEncoder(categories='auto')
    ohe_multi.fit(user_df[my_cols])
    ohe_mat = ohe_multi.transform(user_df[my_cols])

    # Then KMeans cluster
    k_clusters = KMeans(n_clusters=8, random_state=42)
    k_clusters.fit(ohe_mat)

    preds = k_clusters.predict(ohe_mat)
    
    # Add clusters to user df
    user_df['cluster'] = preds
    
    return user_df


In [30]:
test_df = add_clusters_to_users()

/Users/nawjux/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [32]:
test_df.to_csv('data/u_info.csv')

In [ ]:
temp_ohe = ohe_2.get_feature_names(['age'])


In [169]:
gender_df = pd.DataFrame(gender_ohe.toarray(), columns=['F', 'M'])
gender_df.head()

,F,M
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [169]:
ohe.fit(user_df[['gender']])
gender_ohe = ohe.transform(user_df[['gender']])
gender_df = pd.DataFrame(gender_ohe.toarray(), columns=['F', 'M'])
gender_df.head()

,F,M
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [178]:
ohe_2.fit(user_df[['age']])
temp_ohe = ohe_2.get_feature_names(['age'])
age_ohe = ohe_2.transform(user_df[['age']])
age_df = pd.DataFrame(age_ohe.toarray(), columns=temp_ohe)
age_df.head()

,age_1,age_18,age_25,age_35,age_45,age_50,age_56
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [183]:
ohe_3.fit(user_df[['occupation']])
cols = ohe_3.get_feature_names(['occupation'])
occ_ohe = ohe_3.transform(user_df[['occupation']])
occ_df = pd.DataFrame(occ_ohe.toarray(), columns=cols)
occ_df.head()

,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,...,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18,occupation_19,occupation_20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [184]:
all_cat = pd.concat([gender_df, age_df, occ_df], axis=1)

In [185]:
all_cat.head()

,F,M,age_1,age_18,age_25,age_35,age_45,age_50,age_56,occupation_0,...,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18,occupation_19,occupation_20
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [291]:
k_clusters = KMeans(n_clusters=8, random_state=42)

In [292]:
k_clusters.fit(all_cat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0)

In [293]:
preds = k_clusters.predict(all_cat)

In [294]:
preds

array([4, 0, 2, ..., 4, 4, 2], dtype=int32)

In [34]:
user_df['cluster'] = preds

In [35]:
user_df[user_df['id'] == 6040]

,id,gender,age,occupation,cluster
6039,6040,M,25,6,2


In [297]:
cluster_dict = {}
for k, v in zip(user_df['id'].tolist(), user_df['cluster'].tolist()):
    cluster_dict[k] = v

In [298]:
ratings_df['cluster'] = ratings_df['user_id'].apply(lambda x: cluster_dict[x])

In [33]:
def add_cluster_to_ratings(user_df):
    """
    given user_df with clusters, add clusters to ratings data
    parameters
    ---------
    user_df: df with user data

    returns
    -------
    ratings_df: ratings_df with cluster column
    """
    # Read in ratings file
    #Get ratings file
    ratings_df = spark.read.json('data/ratings.json').toPandas()
    
    # Set up clusters
    cluster_dict = {}
    for k, v in zip(user_df['id'].tolist(), user_df['cluster'].tolist()):
        cluster_dict[k] = v
        
    # Add cluster to ratings
    ratings_df['cluster'] = ratings_df['user_id'].apply(lambda x: cluster_dict[x])
    
    return ratings_df

In [15]:
all_df = add_cluster_to_ratings(user_df)

NameError: name 'add_cluster_to_ratings' is not defined

In [42]:
all_df.to_csv('data/user_cluster.csv')

In [39]:
movie_by_cluster = all_df.groupby(by=['cluster', 'movie_id']).agg({'rating': 'mean'}).reset_index()

In [43]:
movie_by_cluster.head()

,cluster,movie_id,rating
0,0,1,3.942857
1,0,2,3.265306
2,0,3,2.756757
3,0,4,3.000000
4,0,5,2.571429


In [8]:
movie_by_cluster = pd.read_csv('data/u_info.csv', index_col=0)
movie_by_cluster.head()

,id,gender,age,occupation,zip,cluster
0,1,F,1,10,48067,4
1,2,M,56,16,70072,0
2,3,M,25,15,55117,2
3,4,M,45,7,02460,1
4,5,M,25,20,55455,2


In [299]:
ratings_df.head()

,movie_id,rating,timestamp,user_id,cluster
0,858,4,956678732.0,6040,2
1,2384,4,956678754.0,6040,2
2,593,5,956678754.0,6040,2
3,1961,4,956678777.0,6040,2
4,1419,3,956678856.0,6040,2


In [300]:
request_df.head()

,movie_id,rating,timestamp,user_id
0,2019,NaN,956678777.0,6040
1,759,NaN,956679248.0,6040
2,2858,NaN,956679275.0,6040
3,246,NaN,956679413.0,6040
4,1617,NaN,956679473.0,6040


In [301]:
def cluster_rating(df, movie_id, cluster):
    cluster_rating = df[(df['movie_id'] == movie_id) & (df['cluster'] == cluster)]
    return cluster_rating['rating'].mean()

def user_bias(df, user_id):
    return  df.loc[df['user_id'] == user_id, 'rating'].mean() - df['rating'].mean()

def item_bias(df, movie_id):
    return  df.loc[df['movie_id'] == movie_id, 'rating'].mean() - df['rating'].mean()

In [281]:
avg = cluster_rating(df=ratings_df, movie_id=1617, cluster=1)

4.056818181818182

In [282]:
u = user_bias(ratings_df, 6040)

-0.0743489043735579

In [283]:
i = item_bias(ratings_df, 2019)

0.9880166870242912

In [279]:
avg + u + i

4.970485964468915

In [280]:
movie_info_df[movie_info_df['id'] == 1617]

,id,name,genres
1575,1617,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller


In [3]:
def get_cold_start_rating(user_id, movie_id):
    """
    Given user_id and movie_id, return a predicted rating
    
    parameters
    ----------
    user_id, movie_id
    
    returns
    -------
    movie rating (float)
    """
    # Get user df with clusters 
    user_df = pd.read_csv('data/user_cluster.csv', index_col=0) 
    u_clusters = pd.read_csv('data/u_info.csv', index_col=0)
 
    # Get ratings data, with clusters
    ratings_df = pd.read_csv('data/movie_cluster_avg.csv', index_col=0)
    
    # User Cluster
    user_cluster = u_clusters.loc[u_clusters['id'] == user_id]['cluster'].tolist()[0]
    
    # Get score components
    avg = ratings_df.loc[ratings_df['user_id'] == movie_id]['rating'].tolist()[0]
    u = user_bias(user_df, user_id)
    i = item_bias(user_df, movie_id)
    
    pred_rating = avg + u + i
    
    return pred_rating
    

In [3]:
blah = get_cold_start_rating(user_id=53, movie_id=1419)

In [4]:
blah

3.692307692307693

In [6]:
df = pd.read_csv('data/user_cluster.csv', index_col=0)

In [3]:
blah is nan

NameError: name 'blah' is not defined

In [3]:
ratings_df = pd.read_csv('data/movie_cluster_avg.csv', index_col=0)
ratings_df.head()

,cluster,movie_id,rating
0,0,1,3.942857
1,0,2,3.265306
2,0,3,2.756757
3,0,4,3.000000
4,0,5,2.571429
